In [11]:
def get_vocab_size(tokenizer):
    vocab=tokenizer.vocab
    special_tokens=tokenizer.special_tokens
    return len(vocab)+len(special_tokens)

In [12]:
#ALL ARE HYPER PARAMETERS. while deploying use mlflow/hyperopt/gridsearch cv
import torch
from minbpe import BasicTokenizer
tokenizer= BasicTokenizer()
torch.manual_seed(3647) #to reproduce same model weights + data shuffling
block_size=256 #no. of tokens it will process each sequence
n_embd=384 #embedding dimension. more dim= can capture more nuanced patterns
n_head=6 #better ability to capture diverse relation
n_layer=6 #6 is good for mid scaled llm
dropout=0.2 # randomly sets 20% of neurons to 0 to overfitting
vocab_size=get_vocab_size(tokenizer)
device= 'cuda' if torch.cuda.is_available() else 'cpu'

In [13]:
from typing import Optional, Tuple
import torch
import torch.nn as nn
from torch.nn import functional as F

class Head(nn.Module):
    def __init__(self, head_size:int):
        super().__init__()
        self.key=nn.Linear(n_embd, head_size, bias=False)
        self.query=nn.Linear(n_embd, head_size, bias=False)
        self.value=nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(
            torch.ones(block_size, block_size))) #This matrix will be used to mask future tokens in causal attention.
        self.dropout=nn.Dropout(dropout) #prevents overfitting

    def forward(self, x: torch.Tensor):
        _, T, _=x.shape
        k=self.key(x)
        q=self.query(x)
        v=self.value(x)
        weights=q@k.transpose(-2,-1)*k.shape[-1]**-0.5
        weights=weights.masked_fill(
            self.tril[:T,:T]==0,float('-inf'))
        weights=F.softmax(weights, dim=1)
        weights=self.dropout(weights)
        out=weights@v
        return out
    


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads:int, head_size: int):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.projection=nn.Linear(head_size*num_heads, n_embd)
        self.dropout=nn.Dropout(dropout)

    def forward(self, x:torch.Tensor):
        out=torch.cat([h(x) for h in self.heads], dim=-1)
        out=self.dropout(self.projection(out))
        return out
    
class FeedForward(nn.Module):
    def __init__(self, n_embd: int):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )
    
    def forward(self,x:torch.Tensor):
        return self.net(x)
    

class Block(nn.Module):
    def __init__(self, n_embd: int, n_head: int):
        super().__init__()
        head_size=n_embd//n_head
        self.self_attention = MultiHeadAttention(n_head, head_size)
        self.feed_forward=FeedForward(n_embd)
        self.layer_norm_1=nn.LayerNorm(n_embd)
        self.layer_norm_2= nn.LayerNorm(n_embd)

    def forward(self, x:torch.Tensor):
        x=x+self.self_attention(self.layer_norm_1(x))
        x=x+self.feed_forward(self.layer_norm_2(x))
        return x
    

In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table=nn.Embedding(block_size, n_embd) #Since attention is orderless, we need these to make the model understand word order
        self.blocks=nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.final_layer_norm=nn.LayerNorm(n_embd)
        self.final_linear_layer=nn.Linear(n_embd, vocab_size)
        self.apply(self.init_weights)

    def init_weights(self, module: nn.Module): #custom init of weights following normal distribution
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, input_tokens:torch.Tensor, targets:Optional[torch.Tensor]=None):
        B,T=input_tokens.shape #---get embedding from here 
        token_embedding=self.token_embedding_table(input_tokens)
        positional_embedding=self.position_embedding_table(torch.arange(T, device=device))
        x=token_embedding+positional_embedding   #get embedding-----to here
        x=self.blocks(x)
        x=self.final_layer_norm(x)
        logits=self.final_linear_layer(x)

        if targets  is None:
            loss=None
        else:
            B,T,C= logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, input_tokens: torch.Tensor, max_new_tokens: int):
        for _ in range(max_new_tokens):
            cropped_input=input_tokens[:, -block_size:] #past window size to reffer the appended tokens
            logits, _ = self(cropped_input)
            logits=logits[:,-1,:]
            probs=F.softmax(logits, dim=-1)
            idx_next=torch.multinomial(probs, num_samples=1)
            input_tokens=torch.cat((input_tokens,idx_next), dim=1)
        return input_tokens

In [21]:
model=GPTLanguageModel()
model=model.to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

10.935808 M parameters


In [23]:
def print_model_structure(model: torch.nn.Module, indent: str=''):
    for name, child, in model.named_children():
        params=sum(p.numel() for p in child.parameters())
        print(f"{indent}|--{name}:{child.__class__.__name__} ({params:,} parameters")
        print_model_structure(child, indent+'|   ')

print_model_structure(model)

|--token_embedding_table:Embedding (98,304 parameters
|--position_embedding_table:Embedding (98,304 parameters
|--blocks:Sequential (10,639,872 parameters
|   |--0:Block (1,773,312 parameters
|   |   |--self_attention:MultiHeadAttention (590,208 parameters
|   |   |   |--heads:ModuleList (442,368 parameters
|   |   |   |   |--0:Head (73,728 parameters
|   |   |   |   |   |--key:Linear (24,576 parameters
|   |   |   |   |   |--query:Linear (24,576 parameters
|   |   |   |   |   |--value:Linear (24,576 parameters
|   |   |   |   |   |--dropout:Dropout (0 parameters
|   |   |   |   |--1:Head (73,728 parameters
|   |   |   |   |   |--key:Linear (24,576 parameters
|   |   |   |   |   |--query:Linear (24,576 parameters
|   |   |   |   |   |--value:Linear (24,576 parameters
|   |   |   |   |   |--dropout:Dropout (0 parameters
|   |   |   |   |--2:Head (73,728 parameters
|   |   |   |   |   |--key:Linear (24,576 parameters
|   |   |   |   |   |--query:Linear (24,576 parameters
|   |   |   |   